# BIA 6304 Text Mining, Fall 2017
## Assignment 3
### Stuent:  Leonardo Ji
### 9/9/2017

**T1.**  Read in or create a data frame with at least one column of text to be analyzed.  This could be the text you used previously or new text. Based on the context of your dataset and the question you want to answer, identify at what processing you think is necessary (stop words, stemming, custom replacement, etc.) Compare the feature space before and after your processing. 

![R-blogger news site](https://www.r-bloggers.com/wp-content/uploads/2016/04/R_02_2016-05-01.png "R-blogger")
I want to use R-blogger news site data from web scrapping technique used in the last class.

In [2]:
from __future__ import division
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import gzip
import math
import numpy as np
import pandas as pd
import requests
import re

pd.set_option('display.max_colwidth', 150) #important for getting all the text

C:\ProgramData\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [55]:
# R-Blogger news
page = requests.get('https://www.r-bloggers.com')

soup = BeautifulSoup(page.text, "html5lib")
#print(type(soup))
#print (soup.prettify)

In [56]:
BSobj = soup.find_all("div",{"class":"twopost"})

headlines = {}
for item in BSobj:
    date = item.find("div",{"class":"date"}).get_text()
    link = item.a["href"]
    author = item.find("a",{"rel":"author"}).get_text()
    excerpt = item.find("p",{"class":"excerpt"}).get_text().replace('\n', '')
    headlines[item.a.get_text()] = {}
    headlines[item.a.get_text()]["category"] = "Post"
    headlines[item.a.get_text()]["author"] = author
    headlines[item.a.get_text()]["date"] = date
    headlines[item.a.get_text()]["link"] = link
    headlines[item.a.get_text()]["excerpt"] = excerpt
    
#print(headlines)

cnndf = pd.DataFrame.from_dict(headlines,orient="index")
#print(cnndf.shape)
cnndf.reset_index(inplace=True)
cnndf.columns = ['Headline', 'Category', 'author', 'Date', 'link', 'excerpt']
textStr = cnndf['excerpt'].values
cnndf.head()

,Headline,Category,author,Date,link,excerpt
0,Announcing lime – Explaining the predictions of black-box models,Post,Data Imaginist - R posts,"September 13, 2017",https://www.r-bloggers.com/announcing-lime-explaining-the-predictions-of-black-box-models/,I’m very pleased to announce that lime has been released on CRAN. lime is animplementation of the model prediction explanation framework described...
1,Applying machine learning algorithms – exercises,Post,Euthymios Kasvikis,"September 15, 2017",https://www.r-bloggers.com/applying-machine-learning-algorithms-exercises/,"INTRODUCTION Dear reader, If you are a newbie in the world of machine learning, then this tutorial is exactly what you need in order to introduce ..."
2,Asset Contribution to Portfolio Volatility,Post,R Views,"September 12, 2017",https://www.r-bloggers.com/asset-contribution-to-portfolio-volatility/,"In our previous portfolio volatility work, we covered how to import stock prices, convert to returns and set weights, calculate portfolio volatili..."
3,Brazilian Yield Curve,Post,R and Finance,"September 14, 2017",https://www.r-bloggers.com/brazilian-yield-curve/,The latest version of GetTDData offers function get.yield.curve todownload...
4,Building rich analytic web apps has never been easier,Post,modern.data,"September 13, 2017",https://www.r-bloggers.com/building-rich-analytic-web-apps-has-never-been-easier/,"We’re entering a golden age of the “analytics developer.” Cutting edge companies like AirBnB, Apple, and S&P are no longer buying off-the-shelf BI..."


* I use tf_idf weight, did not normalize value, used english stop words, set min_df to 5%, set max_df to 50%, ngram 1 to 2. It returned 40 feature spaces.

In [97]:
tfidf1 = TfidfVectorizer(use_idf=True, norm=None, stop_words = "english", min_df = 0.05, max_df = 0.5, ngram_range = (1,2)) #define the transformation
tf1_dm = tfidf1.fit_transform(textStr) #apply the transformation
print(tf1_dm.shape)
org_features = tfidf1.get_feature_names()
tfidfVectororizerTable = pd.DataFrame(tf1_dm.toarray(), columns = tfidf1.get_feature_names())
tfidfVectororizerTable.head()

(28, 40)


,analysis,analytics,announce,arrived,arrived cran,available,blog,collection,cran,data,...,rstudio,set,time,trust,used,ve,version,week,working,world
0,0.0,0.000000,3.268684,0.0,0.0,0.0,0.0,0.0,2.757858,0.0,...,0.0,0.000000,0.0,3.268684,0.0,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,2.981001
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,2.981001,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,2.981001,0.0,0.0,0.000000
4,0.0,3.268684,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


* If we modify the Sklearn stop words to add two stop words then run the same Tfid Vectorizer we get 38 features.

In [204]:
skl_stopwords = text.ENGLISH_STOP_WORDS
print(len(skl_stopwords))
nltk_stopwords = stopwords.words("english")
print(len(nltk_stopwords))

318
153


In [181]:
new_skl_stopwords = set(skl_stopwords)
new_skl_stopwords.update(set(['used', 'like', 'need', 'non']))
print(len(new_skl_stopwords))

322


In [182]:
tfidf1 = TfidfVectorizer(use_idf=True, norm=None, stop_words = new_skl_stopwords, min_df = 0.05, max_df = 0.5, ngram_range = (1,2)) #define the transformation
tf1_dm = tfidf1.fit_transform(textStr) #apply the transformation
print(tf1_dm.shape)
org_features = tfidf1.get_feature_names()
tfidfVectororizerTable = pd.DataFrame(tf1_dm.toarray(), columns = tfidf1.get_feature_names())
tfidfVectororizerTable.head()

(28, 36)


,analysis,analytics,announce,arrived,arrived cran,available,blog,collection,cran,data,...,release,rstudio,set,time,trust,ve,version,week,working,world
0,0.0,0.000000,3.268684,0.0,0.0,0.0,0.0,0.0,2.757858,0.0,...,0.0,0.0,0.000000,0.0,3.268684,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,2.981001
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,2.981001,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,2.981001,0.0,0.0,0.000000
4,0.0,3.268684,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000


* If remove two stop words from NLTK stop words then run Tfid Vectorizer we get 45 features.

In [183]:
a = set(nltk_stopwords)
keepwords = set(['before', 'after'])
# remove before and after from NLTK stop words.
new_nltk_stopwords = a - keepwords
print(len(new_nltk_stopwords))

151


* In the second try I use tf_idf weight, did not normalize value, used modified NLTK stop words, set min_df to 5%, set max_df to 50%, ngram 1 to 2. It returned 45 feature spaces.  Modified NLTK stop words resulted in 5 more features than Sklearn stop words.

In [184]:
tfidf2 = TfidfVectorizer(use_idf=True, norm=None, stop_words = new_nltk_stopwords, min_df = 0.05, max_df = 0.5, ngram_range = (1,2)) #define the transformation
tf2_dm = tfidf2.fit_transform(textStr) #apply the transformation
print(tf2_dm.shape)
tfidfVectororizerTable = pd.DataFrame(tf2_dm.toarray(), columns = tfidf2.get_feature_names())
tfidfVectororizerTable.head()

(28, 45)


,analysis,analytics,announce,around,around world,arrived,arrived cran,available,before,blog,...,release,rstudio,set,time,trust,used,version,week,working,world
0,0.0,0.000000,3.268684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,3.268684,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,2.981001
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.981001,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,2.981001,0.0,0.0,0.000000
4,0.0,3.268684,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [204]:
set(tfidf2.get_feature_names()) - set(tfidf1.get_feature_names())

{'around',
 'around world',
 'before',
 'everyone',
 'get',
 'like',
 'much',
 'need',
 'non',
 'used'}

* Use Porter Stemmer resulted the same feature spaces.

In [185]:
ps = PorterStemmer() #define method (http://www.nltk.org/api/nltk.stem.html#nltk.stem.porter.PorterStemmer)
textStr2 = [ps.stem(word) for word in textStr] 

In [186]:
tf1_dm = tfidf1.fit_transform(textStr2) #apply the transformation
print(tf1_dm.shape)
ps_features = tfidf1.get_feature_names()

(28, 36)


In [187]:
print(len(ps_features))
diff = set(org_features) - set(ps_features)
print(len(diff))

36
0


* Use Lancaster Stemmer resulted the same feature spaces.

In [188]:
ls = LancasterStemmer()
textStr3 = [ls.stem(word) for word in textStr]

In [189]:
tf1_dm = tfidf1.fit_transform(textStr3) #apply the transformation
print(tf1_dm.shape)
ls_features = tfidf1.get_feature_names()

(28, 36)


In [190]:
print(len(ls_features))
diff = set(org_features) - set(ls_features)
print(len(diff))

36
0


* Use Snowball Stemmer resulted the same feature spaces.

In [191]:
ss = SnowballStemmer("english")
textStr4 = [ss.stem(word) for word in textStr]

In [192]:
tf1_dm = tfidf1.fit_transform(textStr4) #apply the transformation
print(tf1_dm.shape)
ss_features = tfidf1.get_feature_names()

(28, 36)


In [193]:
print(len(ss_features))
diff = set(org_features) - set(ss_features)
print(len(diff))

36
0


* Use Lemmatization resulted the same feature spaces.

In [194]:
#lemmatization
wnl = WordNetLemmatizer()
textStr5 = [wnl.lemmatize(word, pos = "v") for word in textStr]

In [195]:
tf1_dm = tfidf1.fit_transform(textStr5) #apply the transformation
print(tf1_dm.shape)
lemm_features = tfidf1.get_feature_names()

(28, 36)


In [196]:
print(len(lemm_features))
diff = set(org_features) - set(lemm_features)
print(len(diff))

36
0


* Define a dictionary of replacement words.

In [197]:
# using dictionaries
news_dict = {'r-studio':'rstudio'}

* Use replace function on words in dictionary on each documents:

In [198]:
newtext = []
for item in cnndf['excerpt']:
    for key, value in news_dict.items():
        item = item.lower().replace(key.lower(), value)
    newtext = newtext + [item]
print(len(newtext))
cnndf['cleantext'] = newtext
cnndf[0:1]

28


,Headline,Category,author,Date,link,excerpt,cleantext
0,Announcing lime – Explaining the predictions of black-box models,Post,Data Imaginist - R posts,"September 13, 2017",https://www.r-bloggers.com/announcing-lime-explaining-the-predictions-of-black-box-models/,I’m very pleased to announce that lime has been released on CRAN. lime is animplementation of the model prediction explanation framework described...,i’m very pleased to announce that lime has been released on cran. lime is animplementation of the model prediction explanation framework described...


* Use regular expression to to accomplish the same replacement task:

In [199]:
def multiple_replace(dict, text): 

  """ Replace in 'text' all occurences of any key in the given
  dictionary by its corresponding value.  Returns the new tring.""" 
  text = str(text).lower()

  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

In [200]:
cnndf['cleantext'] = cnndf['excerpt'].apply(lambda x: multiple_replace(news_dict, x))
cnndf[0:1] 

,Headline,Category,author,Date,link,excerpt,cleantext
0,Announcing lime – Explaining the predictions of black-box models,Post,Data Imaginist - R posts,"September 13, 2017",https://www.r-bloggers.com/announcing-lime-explaining-the-predictions-of-black-box-models/,I’m very pleased to announce that lime has been released on CRAN. lime is animplementation of the model prediction explanation framework described...,i’m very pleased to announce that lime has been released on cran. lime is animplementation of the model prediction explanation framework described...


Custom replacement did not change the number of feature space but it include r-studio and rstudio in the same feature weight number.

In [201]:
tf1_dm = tfidf1.fit_transform(cnndf['cleantext']) #apply the transformation
print(tf1_dm.shape)
clean_text_features = tfidf1.get_feature_names()
diff = set(clean_text_features) - set(org_features)  
diff

(28, 36)


set()

In [202]:
tfidfVectororizerTable = pd.DataFrame(tf1_dm.toarray(), columns = tfidf1.get_feature_names())
tfidfVectororizerTable.head()

,analysis,analytics,announce,arrived,arrived cran,available,blog,collection,cran,data,...,release,rstudio,set,time,trust,ve,version,week,working,world
0,0.0,0.000000,3.268684,0.0,0.0,0.0,0.0,0.0,2.757858,0.0,...,0.0,0.0,0.000000,0.0,3.268684,0.0,0.000000,0.0,0.0,0.000000
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,2.981001
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,2.981001,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,2.981001,0.0,0.0,0.000000
4,0.0,3.268684,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000


**Q1**. Write a short description of the context of the dataset in your own words. Make sure your answer is no longer than three paragraphs, and should at minimum answer these questions:
* Why did you choose the processing that you did? Give several specific examples. 
* What is the effect of the replacement on your feature space?  Does this make sense? Is it helpful for answering your question?  Why or why not?
Audience: technical – fellow data scientists or other technical staff.

I collected twenty-eight blog posts from R-blogger site. The dataset contains five columns: category, author, date, link, excerpt. The excerpt column contains excerpt text that I can create a vector space model. Vector space model convert text documents into numeric model such as a sparse matrix. The Sklearn Learn library vector space function allow me to transform blog posts to a sparse matrix of documents (rows) and features/words (columns).

With all other parameters the same I used Sklearn stop words, a modified version of Sklearn stop words, and modified version of NLTK stop words to create vector space model. Using Sklearn stop words the vector space model contains 40 feature spaces.  Then I added four stop words "used", "like", "need", "non" to the Sklarn stop words set, run the same vector space model, and the feature space number is 36. Using NLTK stop words I actually got 45 feature spaces. Then I compare the difference and see the NLTK stop words list are smaller and it did not filter out some words like "around", "everyone", "much"...etc. Next I applied porter stemmer, Lancaster stemmer, Snowball stemmer, Lemmatization to the text before create vector space model using modified Sklearn stop words.  The results from all four techniques did not change still 36 feature spaces.  Lastly I used regular expression to replace word "r-studio" to "rstudio".

This change make sense because the question I want to ask is which blog post to read to learn r-studio verse all other blog posts. Because "r-studio" and "rstudio" are the same thing I want to add the document frequency to the same feature space, and to the same tfidf weight number. In this case the effect of this replacement did not change the feature space or the "rstudio" column weights, but it makes sense to do this replacement knowing "r-studio" and "rstudio" are referring the same thing and in case in text both these terms occur we want to group them together.  "r-studio" is an R language IDE software. 

**T2.** Create a sentiment dictionary from one of the sources in class or find/create your own (potential bonus points for appropriate creativity). Using your dictionary, create sentiment labels for the text entries in your corpus.

![SNAP](https://snap.stanford.edu/images/snap_logo.png "Stanford Network Analysis Project")
I want to use a Amazon grocery and gourmet food review data from [SNAP](https://snap.stanford.edu/data/web-Amazon.html)

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

amazonFood = getDF('reviews_Grocery_and_Gourmet_Food_5.json.gz')
print(len(amazonFood))
amazonFood.head()

151254


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1VEELTKS8NLZB,616719923X,Amazon Customer,"[0, 0]",Just another flavor of Kit Kat but the taste is unique and a bit different. The only thing that is bothersome is the price. I thought it was a b...,4.0,Good Taste,1370044800,"06 1, 2013"
1,A14R9XMZVJ6INB,616719923X,amf0001,"[0, 1]","I bought this on impulse and it comes from Japan, which amused my family, all those weird stamps and markings on the package. So that was fun. ...",3.0,"3.5 stars, sadly not as wonderful as I had hoped",1400457600,"05 19, 2014"
2,A27IQHDZFQFNGG,616719923X,Caitlin,"[3, 4]",Really good. Great gift for any fan of green tea! Just so expensive to purchase candy from across the sea.,4.0,Yum!,1381190400,"10 8, 2013"
3,A31QY5TASILE89,616719923X,DebraDownSth,"[0, 0]","I had never had it before, was curious to see what it was like. Smooth, great subtle good flavor. I am ordering more and plan to make it a routine.",5.0,Unexpected flavor meld,1369008000,"05 20, 2013"
4,A2LWK003FFMCI5,616719923X,Diana X.,"[1, 2]","I've been looking forward to trying these after hearing about how popular they were in Japan, and among Kit Kat fans as well. I do not recommend o...",4.0,"Not a very strong tea flavor, but still yummy & creamy!",1369526400,"05 26, 2013"


* [AFINN Dictionary](http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6010)

In [5]:
#some dictionaries assign a value 
afinn = {}
for line in open("AFINN-111.txt"):
    tt = line.split('\t')
    afinn.update({tt[0]:int(tt[1])})
print(type(afinn), len(afinn))

<class 'dict'> 2477


In [6]:
#amplification and negation words from qdap
negate = ["aint", "arent","cant", "couldnt" , "didnt" , "doesnt" ,"dont" ,"hasnt" , "isnt" ,"mightnt" , "mustnt" ,"neither" ,"never", "no" ,"nobody" , "nor", "not" , "shant", "shouldnt", "wasnt" , "werent" ,"wont", "wouldnt"]
amplify = ["acute" ,"acutely", "certain", "certainly" ,"colossal", "colossally","deep" , "deeply" , "definite","definitely" ,"enormous","enormously" , "extreme", "extremely" ,"great","greatly" ,"heavily", "heavy", "high","highly" ,"huge","hugely" , "immense", "immensely" ,"incalculable" ,"incalculably","massive", "massively", "more","particular" ,"particularly","purpose", "purposely", "quite" ,"real" ,"really","serious", "seriously", "severe","severely" ,"significant" ,"significantly","sure","surely" , "true" ,"truly" ,"vast" , "vastly" , "very"]


AFINN function to use amplification and negation words from qdap.  The previous amplification doubles the sentiment score while negation reverse the score.

In [111]:
def afinn_sent(inputstring):
    sentcount =0
    i=0
    for word in inputstring.split():
        prev = inputstring.split().pop(i-1)
        if word in afinn:
            if negate.count(prev):
                # negate positive words means negative sentiment score
                sentcount -= afinn[word]
            elif amplify.count(prev):
                # amplification double positive sentiment score
                sentcount += afinn[word]*2
            else:
                sentcount += afinn[word]
        i+=1
    
    if (sentcount < 0):
        sentiment = 'Negative'
    elif (sentcount > 0):
        sentiment = 'Positive'
    else:
        sentiment = 'Neutral'
    
    return sentiment

In [112]:
amazonFood['afinn'] = amazonFood.reviewText.apply(lambda x: afinn_sent(x.lower()))
amazonFood[['reviewText','afinn']].head()

,reviewText,afinn
0,Just another flavor of Kit Kat but the taste is unique and a bit different. The only thing that is bothersome is the price. I thought it was a b...,Negative
1,"I bought this on impulse and it comes from Japan, which amused my family, all those weird stamps and markings on the package. So that was fun. ...",Positive
2,Really good. Great gift for any fan of green tea! Just so expensive to purchase candy from across the sea.,Positive
3,"I had never had it before, was curious to see what it was like. Smooth, great subtle good flavor. I am ordering more and plan to make it a routine.",Positive
4,"I've been looking forward to trying these after hearing about how popular they were in Japan, and among Kit Kat fans as well. I do not recommend o...",Positive


* [UIC Bing Liu Sentiment Lexicon](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html)

In [7]:
#some sort into buckets
HLpos = [line.strip() for line in  open('HLpos.txt','r')]
HLneg = [line.strip() for line in  open('HLneg.txt','r',encoding = 'latin-1')]
print("HL pos  size: " + str(len(HLpos)))
print("HL neg  size: " + str(len(HLneg)))

HL pos  size: 2006
HL neg  size: 4783


HL function also uses amplification and negation words from qdap to calculates the sentiment.
![HL Sentiment Formula](HLFormula.png "HLFormula")
$$Sentiment = \frac{(positiveCount - negativeCount)}{(positiveCount + negativeCount)}$$


In [25]:
def hl_sent(inputstring):

    poscount = 0
    negcount = 0
    i = 0

    for word in inputstring.split():
        if i > 0:
            prev = inputstring.split().pop(i-1)
        else:
            prev =""

        if HLpos.count(word):
            if negate.count(prev):
                negcount += 1
            elif amplify.count(prev):
                poscount +=2
            else: 
                poscount +=1
        elif HLneg.count(word):
            if negate.count(prev):
                poscount += 1
            elif amplify.count(prev):
                negcount +=2
            else:
                negcount +=1
        i+=1
    
    if poscount+negcount > 0:
        t = float((poscount - negcount)/(poscount+negcount))
        
    else:
        t = 0
    
    if t > 0:
        tone = "Positive"
    elif t < 0:
        tone = "Negative"
    else:
        tone = "Neutral"
    
    return tone

In [169]:
amazonFood['hl'] = amazonFood.reviewText.apply(lambda x: hl_sent(x.lower()))
amazonFood[['reviewText','afinn','hl']].head()

,reviewText,afinn,hl
0,Just another flavor of Kit Kat but the taste is unique and a bit different. The only thing that is bothersome is the price. I thought it was a b...,Negative,Negative
1,"I bought this on impulse and it comes from Japan, which amused my family, all those weird stamps and markings on the package. So that was fun. ...",Positive,Neutral
2,Really good. Great gift for any fan of green tea! Just so expensive to purchase candy from across the sea.,Positive,Neutral
3,"I had never had it before, was curious to see what it was like. Smooth, great subtle good flavor. I am ordering more and plan to make it a routine.",Positive,Positive
4,"I've been looking forward to trying these after hearing about how popular they were in Japan, and among Kit Kat fans as well. I do not recommend o...",Positive,Positive


* [NLTK Vader Lexicon](http://www.nltk.org/howto/sentiment.html)

In [135]:
def vader_sent(inputstring):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(inputstring)
    #for k in sorted(ss):
    #    print('{0}: {1}, '.format(k, ss[k]), end='')
    tone = 'Neutral'
    t = ss['pos'] - ss['neg']
    if ss['neu'] > .5:
        tone = 'Neutral'
    elif t > 0:
        tone = "Positive"
    elif t < 0:
        tone = "Negative"
    return tone

In [136]:
amazonFood['vader'] = amazonFood.reviewText.apply(lambda x: vader_sent(x.lower()))
amazonFood[['reviewText','afinn','hl','vader']].head()

,reviewText,afinn,hl,vader
0,Just another flavor of Kit Kat but the taste is unique and a bit different. The only thing that is bothersome is the price. I thought it was a b...,Negative,Negative,Neutral
1,"I bought this on impulse and it comes from Japan, which amused my family, all those weird stamps and markings on the package. So that was fun. ...",Positive,Neutral,Neutral
2,Really good. Great gift for any fan of green tea! Just so expensive to purchase candy from across the sea.,Positive,Neutral,Neutral
3,"I had never had it before, was curious to see what it was like. Smooth, great subtle good flavor. I am ordering more and plan to make it a routine.",Positive,Positive,Neutral
4,"I've been looking forward to trying these after hearing about how popular they were in Japan, and among Kit Kat fans as well. I do not recommend o...",Positive,Positive,Neutral


In [138]:
#save and read to binary file so I donnot have to run again.
amazonFood.to_pickle("amazonFood.bin")
#amazonFood.to_csv("amazonFood.csv")

In [3]:
amazonFood = pd.read_pickle('amazonFood.bin')

* I used a voting system using previous three sentiment results to get final results.  This is like a ensemble model to combine three methods to reduce variance.

In [144]:
def vote_sent(amazonFood):
    vote = [] 
    for index, row in amazonFood.iterrows():
        t = 0
        if (row['afinn'] == 'Positive'):
            t += 1
        elif (row['afinn'] == 'Negative'):
            t -=1
        
        if (row['hl'] == 'Positive'):
            t += 1
        elif (row['hl'] == 'Negative'):
            t -=1    

        if (row['vader'] == 'Positive'):
            t += 1
        elif (row['vader'] == 'Negative'):
            t -=1
    
        if t > 0:
            vote.append('Positive')
        elif t < 0:
            vote.append('Negative')
        else:
            vote.append('Neutral')

    amazonFood['vote'] = vote
  

In [145]:
vote_sent(amazonFood)
amazonFood[['reviewText','afinn','hl','vader','vote']].head()

,reviewText,afinn,hl,vader,vote
0,Just another flavor of Kit Kat but the taste is unique and a bit different. The only thing that is bothersome is the price. I thought it was a b...,Negative,Negative,Neutral,Negative
1,"I bought this on impulse and it comes from Japan, which amused my family, all those weird stamps and markings on the package. So that was fun. ...",Positive,Neutral,Neutral,Positive
2,Really good. Great gift for any fan of green tea! Just so expensive to purchase candy from across the sea.,Positive,Neutral,Neutral,Positive
3,"I had never had it before, was curious to see what it was like. Smooth, great subtle good flavor. I am ordering more and plan to make it a routine.",Positive,Positive,Neutral,Positive
4,"I've been looking forward to trying these after hearing about how popular they were in Japan, and among Kit Kat fans as well. I do not recommend o...",Positive,Positive,Neutral,Positive


In [210]:
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 5) & ('Positive' != amazonFood['hl'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))

10465
0.0691882528726513


In [130]:
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 5) & ('Positive' != amazonFood['vote'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))
ratingDoesnotMatch.head()

9907
0.06549909423883005


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,afinn,hl,vader,vote
44,A3MLKJ1HHA2SX0,B0000531B7,Robert L. Stinnett,"[5, 6]","If you are a fan of peanut butter and chocolate, than this bar is for you. I've tried a number of energy bars and they all suffer from one proble...",5.0,Best Energy Bar Out There,1162598400,"11 4, 2006",Negative,Negative,Neutral,Negative
58,A2B7BUH8834Y6M,B0000537AF,"Shelley Gammon ""Geek""","[1, 1]","I love having one of these in my backpack or purse as a back-up pick-me up that isn't overloaded with sugar, sodium or artificial sweeteners. I'm ...",5.0,My favorite protein bar,1341273600,"07 3, 2012",Negative,Positive,Neutral,Neutral
59,A3NEAETOSXDBOM,B0000537AF,Stephen M. Charme,"[1, 1]",I am a runner and discovered this in a grocery store when I was at the four mile mark of a ten mile run and stopped for some water. This tastes gr...,5.0,Tastes great and gives energy,1219622400,"08 25, 2008",Negative,Positive,Neutral,Neutral
60,A2BB4DGBRVGKWW,B00005C2M2,"A Customer ""customer12345657""","[1, 2]","This is awesome. It tastes like the fake marshmallows they put in immoral breakfast cereal, but instead of just tasting like sugar it tastes like...",5.0,Mmmmmmmm,1351987200,"11 4, 2012",Positive,Negative,Neutral,Neutral
66,A3OFJES10PZGEH,B00005C2M2,SN,"[0, 0]",It's cracked to pieces on purpose so it's easier to eat. It's a bit sweeter than regular ice cream because there's no water. Convenient to take ...,5.0,Great Taste! Convenient! Sweet!,1312848000,"08 9, 2011",Positive,Positive,Neutral,Negative


In [134]:
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 1) & ('Negative' != amazonFood['vote'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))
ratingDoesnotMatch.head()

4067
0.026888545096328032


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,afinn,hl,vader,vote
9,A33NXNZ79H5K51,616719923X,"Jean M ""JM""","[0, 10]","I love green tea, I love Kit Kats, but the two do not belong together. I hate the after taste of them.",1.0,Not a fan,1348012800,"09 19, 2012",Positive,Positive,Neutral,Positive
32,A14YSMLYLJEMET,B00004S1C5,Amazon Customer,"[8, 11]",This product is no where near natural / organic-I only wish I had seen the other reviews before purchasing! It contains all the things I did not w...,1.0,Not natural/organic at all,1364515200,"03 29, 2013",Positive,Positive,Neutral,Positive
75,A3OH4OZFZGEH75,B0000CCZYY,Amazon Customer,"[1, 4]","Licorice is my favorite candy, and it promotes good digestion. Since I have some digestive issues, I eat licorice on a regular basis. I've tried b...",1.0,Not soft at all. Basically same as cheap licorice but with stronger flavor.,1365120000,"04 5, 2013",Positive,Positive,Neutral,Positive
82,A2OUNVRPRWH0,B0000CCZYY,"The Kittie ""Kittie""","[6, 11]","This is an awesome product, natural, not a lot of ingredients, good flavor, not too sweet.But I no longer love our scaredy-cat friends at Amazon! ...",1.0,Love this candy!,1367884800,"05 7, 2013",Positive,Positive,Neutral,Positive
85,AX04H2SPKO02S,B0000CD06J,"J. Wang ""jyswang""","[0, 3]","As soon as I had a couple of sips, my eczema started uncontrollably itching. That's one of the signs when something I eat is NOT gluten-free. Will...",1.0,NOT gluten free,1362528000,"03 6, 2013",Neutral,Neutral,Neutral,Neutral


In [148]:
amazonFood.iloc[96:97][['reviewerID','overall','reviewText','afinn','hl','vader','vote']]

,reviewerID,overall,reviewText,afinn,hl,vader,vote
96,AY8UH4COUYMGZ,5.0,"This bottle is a steal, at Whole Foods (at least where I live) a bottle half this size costs twice as much. Before I was also using imitation vani...",Positive,Negative,Neutral,Neutral


**Q2.** Write a short description of how the sentiment analysis was done and what the outcome is. Make sure your answer is no longer than three paragraphs, and should at minimum answer these questions:
* How did your processing affect the sentiment assignment, if at all?
* What measure did you use to determine the sentiment label?  Why?  Do any of the label assignments surprise you? 
* Include a few specific examples of label assignment and how it was determined and why it does or does not make sense.
Audience: general – management or non-technical staff.

I want to use a different dataset for this question: Amazon grocery and gourmet food review data gathered by Stanford Network Analysis Project (SNAP). This review dataset contains 151,254 user reviews from May 1996 to July 2014.  Dictionary are basically list of words and a sentiment score on each word (positive or negative).   For each document, first I converted text to lower cases, then I matches the positive or negative words from dictionary word list to sum up the total sentiment score.   I tried three different dictionary word list.  The results of all three-sentiment analysis are input into a voting system to determine final sentiment analysis result.  Each method returns a positive, negative or neutral. Which sentiment has the highest voting results is the final sentiment results.  The voting system increased accuracy and reduced variance.  This approach is like an ensemble method.

I was surprised by some of the discrepancies in results. I found out list of reviews where review is negative but user rating is 5/5 (false negative), and another list of reviews where sentiment is positive but user rating is 1/5 (false positive). There are total of 9907 out 151,254 reviews or 6.5% of the reviews are in false negative category, and there are 4067 or 2.7% of the reviews are in false positive category. For example, the review 96 list below [1] is a false negative. The user gives a rating of 5 out of 5 and anyone reads it see writer has a positive sentiment to the product. But there are more negative words than positive thus the negative sentiment result. Similarly review 1334 [2] is a false positive example.  In this example, “pretty” was count as a positive sentiment.  My algorithm did not get it right the first time. In the following section I will discuss in detail how I fixed this problem.

[1] "This bottle is a steal, at Whole Foods (at least where I live) a bottle half this size costs twice as much. Before I was also using imitation vanilla, it smells okay but you can smell the fillers. This bottle smells like fresh, pure vanilla. I used it in a lemon poppy seed cake and it was delicious, mild and aromatic! My kitchen smelled WONDERFUL!"
[2] I have had other stevia products before and liked them. This stuff, however, is incredibly nasty. It tastes chemically and toxic. Twice I tried to get myself to drink it and both times I opted to throw out my coffee rather than tolerate the horrid taste. Awful product.


**T3.** Consider a specific outcome you would like to achieve with your sentiment analysis.  That is, determine what sentiment you might want to have assigned to a specific piece of text.  It could be one entry in your corpus, several documents, or the entire corpus. Make changes to the feature space and/or dictionary to achieve that outcome. Show specific results. 

* I want to print out all negative words in the review.

In [8]:
def hl_output(inputstring):

    poscount = 0
    negcount = 0
    i = 0

    for word in inputstring.split():
        if i > 0:
            prev = inputstring.split().pop(i-1)
        else:
            prev =""

        if HLpos.count(word):
            if negate.count(prev):
                negcount += 1
            elif amplify.count(prev):
                poscount +=2
            else: 
                poscount +=1
        elif HLneg.count(word):
            if negate.count(prev):
                poscount += 1
            elif amplify.count(prev):
                negcount +=2
            else:
                negcount +=1
        i+=1
    
    if poscount+negcount > 0:
        t = float((poscount - negcount)/(poscount+negcount))
        
    else:
        t = 0
    return t
hl_output(amazonFood.iloc[96]['reviewText'].lower())

-0.42857142857142855

* The words "smells", "smell", "smells", "lemon", "smelled" are negative words in HL dictionary. The product rating is 5/5 so this should be a positive review. The term "smelled wonderful!" actual is a positive sentiment. In this case if I first remove punctuations  ("wonderful" instead of "wonderful!") then I can check if the next word is positive or negative word in the HL dictionary. Below is modified version of hl_send function that flip the result from negative to positive.

In [51]:
def hl_sent_mod(inputstring):

    poscount = 0
    negcount = 0
    i = 0
    # Tried different ways 1) remove stop words 2) stemmer 3) remove punctuations.
    #words = [word for word in inputstring.split() if word not in set(skl_stopwords)]
    #ps = PorterStemmer() #define method 
    #words = [ps.stem(word) for word in inputstring] 
    # words = [word for word in inputstring.split()]
    words = [word.replace("!", "").replace(".", "").replace(",", "").replace("(", "").replace(")", "") for word  in inputstring.split()]
    for word in words:
    
        # remove punctuations 
        word = word
        
        if i > 0:
            prev = words[i-1]
        else:
            prev = ""
            
        if i < len(words) - 1:
            nextWord = words[i+1]
        else:
            nextWord = ""
        
        
        if HLpos.count(word):
            if negate.count(prev):
                negcount += 1
            # if next word is negative then this word is actual negative
            elif HLneg.count(nextWord):
                negcount += 1
            elif amplify.count(prev):
                poscount +=2
            else: 
                poscount +=1
        elif HLneg.count(word):
            if negate.count(prev):
                poscount += 1
            # if next word is positive then this word is actual positive
            elif HLpos.count(nextWord):
                poscount += 1
            elif amplify.count(prev):
                negcount +=2
            else:
                negcount +=1
                
        i+=1
    #print(poscount)
    #print(negcount)
    if poscount+negcount > 0:
        t = float((poscount - negcount)/(poscount+negcount))
        
    else:
        t = 0
    
    if t > 0:
        tone = "Positive"
    elif t < 0:
        tone = "Negative"
    else:
        tone = "Neutral"
    
    return tone

In [23]:
print(amazonFood.iloc[96]['reviewText'])

This bottle is a steal, at Whole Foods (at least where I live) a bottle half this size costs twice as much. Before I was also using imitation vanilla, it smells okay but you can smell the fillers. This bottle smells like fresh, pure vanilla. I used it in a lemon poppy seed cake and it was delicious, mild and aromatic! My kitchen smelled WONDERFUL!


In [27]:
hl_sent(amazonFood.iloc[96]['reviewText'].lower())

'Negative'

In [26]:
hl_sent_mod(amazonFood.iloc[96]['reviewText'].lower())

'Positive'

In [52]:
print(amazonFood.iloc[1334]['reviewText'])

I have had other stevia products before and liked them. This stuff, however, is incredibly nasty. It tastes chemically and toxic. Twice I tried to get myself to drink it and both times I opted to throw out my coffee rather than tolerate the horrid taste. Awful product.


In [43]:
hl_sent(amazonFood.iloc[1334]['reviewText'].lower())

'Positive'

In [53]:
hl_sent_mod(amazonFood.iloc[1334]['reviewText'].lower())

'Negative'

* If I re-run all reviews with hl_sent_mod function instead of hl_sent_function. Then calculate the vote results to compare with previous result. We can see the false negative dropped from 6.9% to 5.3%; and the false positive dropped from 2.5% to 2.2%.  

In [11]:
amazonFood['hlMod'] = amazonFood.reviewText.apply(lambda x: hl_sent_mod(x.lower()))

In [12]:
amazonFood[['reviewText','hl','hlMod']].head()

,reviewText,hl,hlMod
0,Just another flavor of Kit Kat but the taste is unique and a bit different. The only thing that is bothersome is the price. I thought it was a b...,Negative,Negative
1,"I bought this on impulse and it comes from Japan, which amused my family, all those weird stamps and markings on the package. So that was fun. ...",Neutral,Positive
2,Really good. Great gift for any fan of green tea! Just so expensive to purchase candy from across the sea.,Neutral,Positive
3,"I had never had it before, was curious to see what it was like. Smooth, great subtle good flavor. I am ordering more and plan to make it a routine.",Positive,Positive
4,"I've been looking forward to trying these after hearing about how popular they were in Japan, and among Kit Kat fans as well. I do not recommend o...",Positive,Positive


In [14]:
amazonFood.to_pickle("amazonFood.bin")
amazonFood.to_csv("amazonFood.csv")
#amazonFood = pd.read_pickle('amazonFood.bin')

In [16]:
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 5) & ('Positive' != amazonFood['hl']) & ('Positive' == amazonFood['hlMod'])]
ratingDoesnotMatch.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,afinn,hl,vader,vote,hlMod
39,A2BYV7S1QP2YIG,B0000531B7,"Dr. Oceanfront ""Oceanfront""","[1, 1]","Frankly, I was worried about buying these on line because I have heard stories about getting expired food. However, these came in and were very fr...",5.0,"Good, filling bar...",1397088000,"04 10, 2014",Positive,Neutral,Neutral,Positive,Positive
61,A14Z6D3IRJ23F7,B00005C2M2,"Chris F ""root""","[0, 0]","I bought these for a lightweight backpacking snack. From what I remembered as a kid they are very tasty, and I was not disappointed! I've read oth...",5.0,Awesome light weigh snack!,1346112000,"08 28, 2012",Positive,Neutral,Neutral,Positive,Positive
96,AY8UH4COUYMGZ,B0000CDEPD,ttt,"[1, 2]","This bottle is a steal, at Whole Foods (at least where I live) a bottle half this size costs twice as much. Before I was also using imitation vani...",5.0,The Only Vanilla I'll Buy From Now On!,1373673600,"07 13, 2013",Positive,Negative,Neutral,Neutral,Positive
115,AI0BCEWRE04G0,B0000CFLIL,Stoney,"[6, 7]","I've used the Mellita Coffee System for about 40 years. I find the system more convenient than coffee machines.If the power goes off, you can sti...",5.0,The Mellita Coffee System,1262822400,"01 7, 2010",Positive,Neutral,Neutral,Neutral,Positive
143,A3JV071KO43Q5X,B0000CNU15,"Jeffrey Pittman ""Jeff""","[8, 9]",My favorite nearby Asian food mart is closing with plans to reopen months from now in a new location. I need this stuff badly as a staple that mu...,5.0,"Great stuff, but just STUPID expensive here unfortunately",1338163200,"05 28, 2012",Negative,Negative,Neutral,Negative,Positive


In [46]:
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 1) & ('Negative' != amazonFood['hl']) & ('Negative' == amazonFood['hlMod'])]
ratingDoesnotMatch.head()

Int64Index([   354,    355,    361,    470,    544,    565,    613,    757,
               827,   1256,
            ...
            147856, 147963, 148479, 148810, 149402, 149632, 149693, 150438,
            150788, 150968],
           dtype='int64', length=675)

In [225]:
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 5) & ('Positive' != amazonFood['hl'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 1) & ('Negative' != amazonFood['hl'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))

10465
0.0691882528726513
3835
0.02535470136326973


In [226]:
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 5) & ('Positive' != amazonFood['hlMod'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 1) & ('Negative' != amazonFood['hlMod'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))

8118
0.05367130786623825
3354
0.022174620175334205


* Recall I used three different dictionaries and use a voting system to get final results.  Use the new hl_sent_mod function also improved the voting result.  It reduced the false negative from 6.5% to 5.0%, and false positive from 2.7% to 2.6%.  The new hl_sent_mod function improves the accuracy judging using all 151,254 Amazon food reviews.

In [227]:
vote_sent(amazonFood)
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 5) & ('Positive' != amazonFood['vote'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))
ratingDoesnotMatch = amazonFood[(amazonFood['overall'] == 1) & ('Negative' != amazonFood['vote'])]
print(len(ratingDoesnotMatch))
print(len(ratingDoesnotMatch)/len(amazonFood))

7626
0.050418501328890475
3982
0.026326576487233397


**Q3.**  Write a short description of the exercise and the outcome.  Make sure your answer is no longer than three paragraphs, and should at minimum answer these questions:
* What outcome did you choose?  Why?
* How did you change the dictionary to achieve that outcome?
* How would you explain (justify, rationalize) those changes if necessary?
Audience: general – management or non-technical staff.

The HL function gives a negative sentiment on review 96. However, the user's product rating is 5/5 so this should be a positive review.  The review is positive from reading it.  But computer does not understand that.  So, I want to change the algorithm to fix it.  It turns out words "smell" "smells", "smell", "smells", "smelled" are all negative words in the HL dictionary. But this is a food review therefore "smell" is a normal word.  And in actual text "smelled wonderful", "smelled okay" are positive or at least neutral. First I realized “smelled wonderful!” did not count because the punctuation was not removed.  By removing the punctuations from "delicious," "aromatic!" and "wonderful!", these words will be matched with positive words in the dictionary, this improves review sentiment to more positive.  Then, we would not miss some important positive or negative words.  But the text still has more negative words than positive words.  At first I wanted to remove "smell" all other "smells", "smell", "smells", "smelled" from the HL dictionary but I decided to not to do it.  This is because "smelled" can still be a negative sentiment in other cases.  For example, "something smelled" like "something stinks" has a negative connotation. I do not want to change the rule here for one text.

I thought we were already looking at previous word why not look at next word too.  By looking at both previous and next words we can get more accurate results.  For words that are both positive and negative sentiment, if the next word is opposite sentiment, then we do not count it.  I modified the function to look at the next word.  After making this change this review’s sentiment result changed from negative to positive.  But this is just one example, how can I justify making a change without testing it on more data?  I can justify the changes after seeing the reductions in false negative and false positive numbers from validation on all 151,254 reviews.  It reduced the false negative from 6.9% to 5.3%, and false positive from 2.5% to 2.2%.  The improvement is because false positive term “smelled wonderful” is positive even though “smelled” is a negative word; false negative term “incredibly nasty” is now negative even though “incredible” is a positive word.  The improvement on false positive is less because there are less terms like “incredibly nasty”, “pretty dreadful”, and “super disgusting” (positive word with next word is negative).  

We only look at one previous word, in the future I want to look at two previous words: “no longer”, “no more”.  “no longer good” should be a negative sentiment.  There are many two words term we can consider.  I did not include them in my function but it certainly can improve result even more.

**Q4.** Data science is all about finding patterns in the data.  You have just been asked to decide on a pattern before finding it. Write a short description of how the easy or difficult it was to arrive at a predetermined conclusion.  How difficult was it to justify? What are the ethical issues involved, if any? What is your role as a data scientist? 
Audience: general – management or non-technical staff.

I can see it would be an ethical concern. In text mining, it is easy to change the dictionary, replace the text, or add or remove or use different list of stop words.  The decision would affect the outcome easily.  Computer does not understand the text, it only understands what human tells it what to do using an algorithm.  The difficult part is to do the right thing and have good rational on making a change.  

It depends on the situation on how to tweak the rules. In my view a data scientist must consider carefully when changing a rule or data.  First, we need to ask a lot of questions first.  Is this a very specific case or unique term and cannot be used in other way?  For example, “NYC” replaces “New York City”, or “BofA” for “Bank of America”.  However, if this change has broad implications in general situations, then we must find a good rule that fit the most of the cases.  If there is less data, we must gather larger data set. The results must be validated using training and validation data.  

We also want to look at more existing research and find whether the pattern is a good fixture based on research.  I can see domain knowledge is also important.  The text documents on financial reporting is very different from movie reviews.  So, I would think as data scientists we need to research domain specific knowledge before making a change and arriving a conclusion.  In BI class we talked more about this, [Spurious correlations site](http://www.tylervigen.com/spurious-correlations) has many examples of incorrect patterns to avoid.